# PSD Module Execution

This notebook runs `pores_analysis.compute_psd` on a segmented volume and exports the resulting PSD table for further analysis.

## Environment Setup

Install the dependencies required to run the PSD pipeline in Colab.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import subprocess

# 1. הסרת חבילות מיותרות (רק אם הן קיימות ועלולות להתנגש)
# אין צורך להתקין מחדש numpy/pandas/matplotlib כי הן מובנות ומעודכנות בקולאב

def setup_cupy():
    try:
        # בדיקת גרסת ה-CUDA שמותקנת כרגע על המכונה
        cuda_version = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
        if "release 12" in cuda_version:
            target_cupy = "cupy-cuda12x"
        elif "release 11" in cuda_version:
            target_cupy = "cupy-cuda11x"
        else:
            target_cupy = "cupy-wheel" # ברירת מחדל חכמה

        print(f"Detected CUDA version. Installing {target_cupy}...")

        # התקנה שקטה רק של מה שצריך
        subprocess.run(["pip", "uninstall", "-y", "cupy-cuda11x", "cupy-cuda12x"], capture_output=True)
        subprocess.run(["pip", "install", target_cupy], capture_output=True)

        import cupy as cp
        print(f"Successfully installed and imported CuPy! GPU device: {cp.cuda.Device(0)}")

    except Exception as e:
        print(f"Setup failed: {e}. Trying generic cupy-wheel...")
        os.system("pip install cupy-wheel")

setup_cupy()

Detected CUDA version. Installing cupy-cuda12x...


## Imports

Import the modules used by the PSD pipeline.

In [3]:
%cd /content/drive/MyDrive/soil_microCT_images/drive_scripts/

/content/drive/MyDrive/soil_microCT_images/drive_scripts


In [4]:
import numpy as np
from pathlib import Path
from pores_analysis import compute_psd, psd_to_dataframe, save_psd_dataframe
from pores_analysis.config_loader import load_config

## Configuration

Define the paths and parameters that are passed directly to the PSD module.

In [19]:
# config_overrides = {
#     "paths": {
#         "input_volume_relative_path": "ROI/z_stability_results/rehovot_ROI_8bit_preproc_masks/binary_pores.npy"
#     },
#     "image_params": {
#         "voxel_spacing": [2.0, 1.0, 1.0]
#     }
# }

config = load_config()  # Override settings per run

volume_path = config["paths"]["input_volume_path"]
voxel_spacing = tuple(config["image_params"]["voxel_spacing"])
use_gpu = config["processing_thresholds"]["use_gpu"]
results_csv_path = Path(config["paths"]["output_dir"]) / "psd_results.csv"
min_reliable_voxels = config["processing_thresholds"]["min_reliable_diameter_voxels"]
print(f'input path: {volume_path}')

input path: /content/drive/MyDrive/soil_microCT_images/ROI/z_stability_results/bnei_reem/binary_pores.npy


In [ ]:
if volume_path.startswith("TODO"):
    raise RuntimeError("TODO: set volume_path to a segmented volume file before running this notebook.")

## Module Execution

Load the volume, compute the PSD, convert it to a DataFrame, and save the table.

In [20]:
print("Loading segmented volume...")
volume = np.load(volume_path)
print(f"Volume shape: {volume.shape}, dtype: {volume.dtype}")
print("Ensuring boolean type...")
volume = volume.astype(bool)
print("Computing PSD...")
psd_result = compute_psd(volume, voxel_spacing=voxel_spacing, use_gpu=use_gpu)
print("Converting PSD to DataFrame...")
psd_df = psd_to_dataframe(psd_result)
print(f"Saving PSD table to {results_csv_path}")
save_psd_dataframe(psd_df, results_csv_path)
print("PSD module execution complete.")

Loading segmented volume...
Volume shape: (100, 650, 650), dtype: uint8
Ensuring boolean type...
Computing PSD...
PSD Calculator - Pipeline Execution
Volume shape: (100, 650, 650)
Voxel spacing (physical): (15.0, 15.0, 15.0) μm
Porosity: 0.2202
GPU: True, Chunking: False

[1/3] Computing Euclidean Distance Transform...
  EDT complete. Max distance: 63.06 voxels

[2/3] Computing Opening Map (Morphological Reconstruction)...
  Computing Opening Map (GPU Iterative, Max Radius: 63)...
    Progress: 10/63 radii processed
    Progress: 20/63 radii processed
    Progress: 30/63 radii processed
    Progress: 40/63 radii processed
    Progress: 50/63 radii processed
    Progress: 60/63 radii processed
  Opening complete. Max diameter: 126.00 voxels

[3/3] Extracting Pore Size Distribution...

PSD Calculation Complete
Total pore voxels analyzed: 9,078,306
Diameter range: 31.33 - 1994.51 μm
Reliable bins (d >= 5 voxels): 39/50
Converting PSD to DataFrame...
Saving PSD table to /content/drive/MyDr

## Outputs / Results

Review the exported PSD data to verify the run.

In [ ]:
print("PSD DataFrame head:")
print(psd_df.head())
reliable_count = psd_df['is_reliable'].sum()
total_bins = len(psd_df)
print(f"Reliable bins (d >= {min_reliable_voxels} voxels): {reliable_count}/{total_bins}")

## Notes / Limitations

- The input volume must be a pre-segmented 3D boolean array with `True` representing pore space, as required by the PSD pipeline.
- Vogel et al. reliability is controlled by `processing_thresholds.min_reliable_diameter_voxels` (default 5 voxels); filter on `is_reliable` before downstream use.
- Border-touching pores are excluded automatically, so ensure the provided volume bounds are acceptable for the intended analysis.